In [16]:
import os
import pandas as pd
from textblob import *
import nltk
import numpy as np
from sklearn import tree
import openpyxl
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import *


training = pd.read_csv(r"data/FinNews.csv", encoding="latin-1")


#remove nonASCIICharacters
training["text"] = training.apply(lambda x: "".join([i for i in x["text"] if ord(i) <= 127]), axis=1)
training






,sentiment,text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...
...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...
4844,negative,Net sales of the Paper segment decreased to EU...


In [17]:
def to_class(s):
    match s:
        case "positive":
            return 2
        case "negative":
            return 1
        case "neutral":
            return 0
    

In [18]:
training["sentiment"] = training.apply(lambda x: to_class(x["sentiment"]), axis=1)

In [19]:


training["text_tb_pol"] = training.apply(lambda x: TextBlob(x["text"]).polarity, axis=1)
training["text_tb_sub"] = training.apply(lambda x: TextBlob(x["text"]).subjectivity, axis=1)





In [20]:
analyzer = SentimentIntensityAnalyzer()

training["text_vader_scores"] = training.apply(lambda x: analyzer.polarity_scores(x["text"]), axis=1)
training["text_vader_neg"] = training.apply(lambda x: x["text_vader_scores"]["neg"], axis=1)
training["text_vader_neu"] = training.apply(lambda x: x["text_vader_scores"]["neu"], axis=1)
training["text_vader_pos"] = training.apply(lambda x: x["text_vader_scores"]["pos"], axis=1)
training = training.drop(["text_vader_scores"], axis=1)








In [21]:
training

,sentiment,text,text_tb_pol,text_tb_sub,text_vader_neg,text_vader_neu,text_vader_pos
0,0,"According to Gran , the company has no plans t...",0.000000,0.000000,0.082,0.855,0.063
1,0,Technopolis plans to develop in stages an area...,0.083333,0.066667,0.068,0.932,0.000
2,1,The international electronic industry company ...,0.000000,0.166667,0.000,1.000,0.000
3,2,With the new production plant the company woul...,-0.064802,0.438695,0.037,0.668,0.295
4,2,According to the company 's updated strategy f...,0.000000,0.000000,0.000,0.876,0.124
...,...,...,...,...,...,...,...
4841,1,LONDON MarketWatch -- Share prices ended lower...,-0.500000,0.300000,0.255,0.677,0.068
4842,0,Rinkuskiai 's beer sales fell by 6.5 per cent ...,0.000000,0.000000,0.000,1.000,0.000
4843,1,Operating profit fell to EUR 35.4 mn from EUR ...,0.000000,0.000000,0.000,0.769,0.231
4844,1,Net sales of the Paper segment decreased to EU...,0.040000,0.330000,0.000,0.937,0.063


In [ ]:
'''
    Begin logistic regression
'''
'''
solver          penalty
lbfgs           l2
newton-cg       l1, l2
sag             l2
saga            elasticnet, l1, l2


'''

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

classes = training["sentiment"].to_numpy()
#features = training[["text_tb_pol", "text_tb_sub", "hash_tb_pol", "hash_tb_sub", "text_vader_neg", "text_vader_neu", "text_vader_pos", "hash_vader_neg", "hash_vader_neu", "hash_vader_pos"]].to_numpy()

'''text only'''
features = training[["text_tb_pol", "text_tb_sub", "text_vader_pos", "text_vader_neg", "text_vader_neu"]].to_numpy()

#validate_features = testing[["text_tb_pol", "text_tb_sub", "hash_tb_pol", "hash_tb_sub", "text_vader_neg", "text_vader_neu", "text_vader_pos", "hash_vader_neg", "hash_vader_neu", "hash_vader_pos"]].to_numpy()


#for i in range(100):

averages = []

total = 0
length = 0

#lbgfs
for i in range(100):
    x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=i)
    lbgfs = LogisticRegression(penalty="l2", solver="lbfgs")
    lbgfs.fit(x_train, y_train)
    results = lbgfs.predict(x_test)
    actual = y_test
    total += (len([i for i in zip(results, actual) if i[0] == i[1]]) / len(actual))
    length += 1
averages.append(total / length)

''''''
total = 0
length = 0
#newton-cg
for i in range(100):
    x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=i)
    newton_cg = LogisticRegression(penalty="l2", solver="newton-cg")
    newton_cg.fit(x_train, y_train)
    results = newton_cg.predict(x_test)
    actual = y_test
    total += (len([i for i in zip(results, actual) if i[0] == i[1]]) / len(actual))
    length += 1
averages.append(total / length)

total = 0
length = 0
#sag
for i in range(100):
    x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=i)
    sag = LogisticRegression(penalty="l2", solver="sag", random_state=i)
    sag.fit(x_train, y_train)
    results = sag.predict(x_test)
    actual = y_test
    total += (len([i for i in zip(results, actual) if i[0] == i[1]]) / len(actual))
    length += 1
averages.append(total / length)

total = 0
length = 0
#saga
for i in range(100):
    x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=i)
    saga = LogisticRegression(penalty="l2", solver="saga", random_state=i)
    saga.fit(x_train, y_train)
    results = saga.predict(x_test)
    actual = y_test
    total += (len([i for i in zip(results, actual) if i[0] == i[1]]) / len(actual))
    length += 1
averages.append(total / length)


averages





[0.6027647867950479,
 0.6027372764786795,
 0.6027510316368637,
 0.6027647867950481]

In [23]:
from sklearn.metrics import confusion_matrix
matrices = []
for i in range(100):
    x_train, x_test, y_train, y_test = train_test_split(features, classes, test_size=0.15, random_state=i)
    lbgfs = LogisticRegression(penalty="l2", solver="lbfgs")
    lbgfs.fit(x_train, y_train)
    results = lbgfs.predict(x_test)
    actual = y_test
    matrices.append(confusion_matrix(y_test, results))

mean_matrix = np.mean(np.array(matrices), axis=0)
mean_matrix

array([[395.66,   8.36,  27.09],
       [ 81.47,   7.91,   2.13],
       [167.73,   2.01,  34.64]])